<a href="https://colab.research.google.com/github/e-olang/Semantic-Search/blob/main/Similarity%20(Semantic%20Search)/embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
torch.cuda.get_device_name(0)

'Tesla T4'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/NLP/Semantic Search

/content/drive/MyDrive/Colab Notebooks/NLP/Semantic Search


In [4]:
path = 'arXiv.csv'

In [8]:
!pip install sentence-transformers -q
!pip install nltk -q
import nltk
nltk.download("stopwords")

     |████████████████████████████████| 85 kB 3.7 MB/s 
     |████████████████████████████████| 5.5 MB 13.3 MB/s 
     |████████████████████████████████| 1.3 MB 46.9 MB/s 
     |████████████████████████████████| 182 kB 52.6 MB/s 
     |████████████████████████████████| 7.6 MB 43.3 MB/s 


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [10]:
x = pd.read_csv(path)
x.head()

,title,first_author,summary,year,month
0,Uni-Perceiver v2: A Generalist Model for Large...,Jifeng Dai,Despite the remarkable success of foundation m...,2022,11
1,Mapping Tropical Forest Cover and Deforestatio...,Sassan Saatchi,Monitoring changes in tree cover for rapid ass...,2022,11
2,Exact Quantum Algorithms for Quantum Phase Rec...,Soonwon Choi,We explore the relationship between renormaliz...,2022,11
3,Machine Learned Calabi--Yau Metrics and Curvature,Justin Tan,Finding Ricci-flat (Calabi--Yau) metrics is a ...,2022,11
4,InstructPix2Pix: Learning to Follow Image Edit...,Alexei A. Efros,We propose a method for editing images from hu...,2022,11


In [11]:
print(x.shape)

(28000, 5)


In [12]:
#x = x[x['title'].str.split().str.len() > 10]        # remove short titles (less than 10 words)

x['clean'] = x['title'].str.replace('\d+', '')
x['clean'] = x['clean'].str.replace('[^\w\s]', '')
x['clean'] = x['clean'].str.lower()

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

x['clean'] = x['clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
x.tail()

,title,first_author,summary,year,month,clean
27995,Efficient Neuromorphic Signal Processing with ...,Mike Davies,The biologically inspired spiking neurons used...,2021,11,efficient neuromorphic signal processing loihi
27996,An Algorithmic Theory of Metacognition in Mind...,Rylan Schaeffer,Humans sometimes choose actions that they them...,2021,11,algorithmic theory metacognition minds machines
27997,Increasing Data Diversity with Iterative Sampl...,Sinan Altinuc,"As a part of the Data-Centric AI Competition, ...",2021,11,increasing data diversity iterative sampling i...
27998,Sharp Bounds for Federated Averaging (Local SG...,Tengyu Ma,"Federated Averaging (FedAvg), also known as Lo...",2021,11,sharp bounds federated averaging local sgd con...
27999,Toward Learning Human-aligned Cross-domain Rob...,Eric Xing,Machine learning has demonstrated remarkable p...,2021,11,toward learning humanaligned crossdomain robus...


In [13]:
# reset index
x = x.reset_index(drop=True)
x.tail()

,title,first_author,summary,year,month,clean
27995,Efficient Neuromorphic Signal Processing with ...,Mike Davies,The biologically inspired spiking neurons used...,2021,11,efficient neuromorphic signal processing loihi
27996,An Algorithmic Theory of Metacognition in Mind...,Rylan Schaeffer,Humans sometimes choose actions that they them...,2021,11,algorithmic theory metacognition minds machines
27997,Increasing Data Diversity with Iterative Sampl...,Sinan Altinuc,"As a part of the Data-Centric AI Competition, ...",2021,11,increasing data diversity iterative sampling i...
27998,Sharp Bounds for Federated Averaging (Local SG...,Tengyu Ma,"Federated Averaging (FedAvg), also known as Lo...",2021,11,sharp bounds federated averaging local sgd con...
27999,Toward Learning Human-aligned Cross-domain Rob...,Eric Xing,Machine learning has demonstrated remarkable p...,2021,11,toward learning humanaligned crossdomain robus...


In [14]:
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')
# print moel parameters
print(model)

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.5k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/383 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)


In [15]:
# add embeddings to dataframe
x['embeddings'] = x['clean'].apply(lambda x: model.encode(x, device='cuda:0'))

In [16]:
x.head()

,title,first_author,summary,year,month,clean,embeddings
0,Uni-Perceiver v2: A Generalist Model for Large...,Jifeng Dai,Despite the remarkable success of foundation m...,2022,11,uniperceiver v generalist model largescale vis...,"[0.014458368, 0.01831729, -0.01943984, -0.0345..."
1,Mapping Tropical Forest Cover and Deforestatio...,Sassan Saatchi,Monitoring changes in tree cover for rapid ass...,2022,11,mapping tropical forest cover deforestation pl...,"[0.018853571, -0.0035001582, 0.019950224, 0.03..."
2,Exact Quantum Algorithms for Quantum Phase Rec...,Soonwon Choi,We explore the relationship between renormaliz...,2022,11,exact quantum algorithms quantum phase recogni...,"[-0.06953129, 0.022138515, -0.057299484, -0.07..."
3,Machine Learned Calabi--Yau Metrics and Curvature,Justin Tan,Finding Ricci-flat (Calabi--Yau) metrics is a ...,2022,11,machine learned calabiyau metrics curvature,"[0.015547868, 0.0100502595, -0.013799696, -0.0..."
4,InstructPix2Pix: Learning to Follow Image Edit...,Alexei A. Efros,We propose a method for editing images from hu...,2022,11,instructpixpix learning follow image editing i...,"[0.025180671, 0.044556007, 0.03460069, 0.00936..."


In [17]:
x.to_csv('titles_embeddings.csv', index=False)

In [18]:
titles = x['title'].tolist()
embeddings = x['embeddings'].tolist()
authors = x['first_author'].tolist()
summaries = x['summary'].tolist()
years = x['year'].tolist()

print(len(titles))
print(len(embeddings))
print(len(authors))
print(len(summaries))
print(len(years))

28000
28000
28000
28000
28000


In [19]:
import pickle
with open('titles.pkl', 'wb') as f:
    pickle.dump(titles, f)

with open('embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

with open('authors.pkl', 'wb') as f:
    pickle.dump(authors, f)

with open('summary.pkl', 'wb') as f:
    pickle.dump(summaries, f)

with open('years.pkl', 'wb') as f:
    pickle.dump(years, f)

In [ ]:
# load titles and embeddings from disk 
with open('titles.pkl', 'rb') as f:
    ltitles = pickle.load(f)
    print(type(titles))
    print(len(titles))

with open('embeddings.pkl', 'rb') as f:
    lembeddings = pickle.load(f)
    print(type(embeddings))
    print(len(embeddings))

<class 'list'>
24469
<class 'list'>
24469


In [ ]:
df2 = pd.DataFrame({'titles': ltitles, 'embeddings': lembeddings})
df2.head()

,titles,embeddings
0,Logic Programming in Assumption-Based Argument...,"[-0.023969512, 0.010221153, 0.0005422938, -0.0..."
1,Modeling and Mining Spatiotemporal Patterns of...,"[0.0034898794, -0.0044532777, -0.053476732, -0..."
2,Mitosis Detection in Breast Cancer Histology I...,"[0.013757976, -0.00069692935, 0.022236006, -0...."
3,Towards Population Scale Activity Recognition:...,"[0.06615189, 0.014459885, -0.035167716, -0.081..."
4,A Platform to Evaluate the Technology for Serv...,"[-0.0020297964, -0.03787091, -0.046355557, -0...."
